# 侧位片分析调用样例

In [ ]:
# 加载所需要的包
import os
import cv2
import glob
import base64
import time
import requests
import json
import urllib
import numpy as np
import matplotlib.pyplot as plt

<h2>定义调用规则</h2>
请根据您从我方获取的信息修改以下代码块

In [ ]:
# 朝厚服务请求地址，随api文档发送
base_url = "<服务请求地址>"

# 朝厚文件服务地址，随api文档发送
file_server_url = "<服务文件服务器地址>"

# 必须传入鉴权 Header。请保护好TOKEN!!! 如果泄露请立即联系我们重置，所有使用该TOKEN的任务都会向您的账户计费
zh_token = "<贵司服务Token, 随合同发送>" # 调用所有的API都必须传入token用作鉴权

user_group = "APIClient" # 用户组，一般为 APIClient

# 贵司user_id, 随api文档发送
user_id = "<贵司user_id>"

# 如果您收到了creds.json, 下面将直接读取
if os.path.exists('../../creds.json'):
    creds = json.load(open('../../creds.json', 'r'))
    base_url = creds['base_url']
    file_server_url = creds['file_server_url']
    zh_token = creds['zh_token']
    user_id = creds['user_id']
    print("loaded creds from creds.json")

In [ ]:
json_call = {
    "spec_version": "1.0-snapshot",
    "spec_name": "ceph-analysis",
    "spec_group": "ceph",
    "user_group": user_group,
    "user_id": user_id
}        

### 定义回调地址

回调会在工作流运行结束后向指定地址发送一个POST请求，请求内容为一个JSON，内含4个字段

1. workflow_id (str): 对应的workflow_id
2. metadata (dict): 您启动工作流时传入的metadata
3. success (bool): 工作流是否成功（true为成功）
4. reason (str or null): 如果success为true, 该项为null. 否则为失败原因的字符串表示

如果需要回调，请uncomment以下代码块

In [ ]:
# json_call['metadata'] = { #（可选）这里可以加入metadata, 回调时会附带该信息，字典每项限制128个字符
#     "case_id": "CH-123",
#     "case_name": "ABCDE"
# }
# json_call['notification'] =[ #（可选）这里可以加入回调地址
#     {"url": "https://www.baidu.com"} # 这里可以加入多个回调地址，每个都是{"url": "xxx"}格式
# ]

## 定义输入/输出块

输入块由JSON文件中的`input_data`定义。对于二维图片，我们支持以下**输入**文件格式

```
"jpg"
"jpeg"
"png"
```

对于二维图片，输入可以采用两种形式传入：

1. 先将文件传入我们的文件服务系统，再将文件指针写入JSON调用串并调用API
2. 直接在API的JSON调用串中传入base64编码后的二进制数据

我们**强烈推荐**使用第一种方式，理由如下：

1. base64编码将增大数据流大小，提高网络延迟
2. 为保障接口性能，朝厚将拒绝过大的API请求。因此对于大文件，第二种方式会调用失败（HTTP CODE 413)

In [ ]:
def upload_file(file_name):
    ext = file_name.split('.')[-1]
    data = open('../../data/' + file_name, 'rb').read()
    resp = requests.get(file_server_url + f"/scratch/{user_group}/{user_id}/upload_url?" +
                        f"postfix={ext}", # 必须指定 postfix, 即文件后缀名
                        headers={"X-ZH-TOKEN": zh_token}) # 获取带签名的上传地址
    resp.raise_for_status()

    upload_url = resp.text[1:-1] # 返回为一个单字符串JSON "string", 这里也可以用json.loads(resp.text)

    resp = requests.put(upload_url, data) # 上传至云储存服务不需要带鉴权头

    resp.raise_for_status()
    path = "/".join(urllib.parse.urlparse(upload_url).path.lstrip("/").split("/")[3:])
    urn = f"urn:zhfile:o:s:{user_group}:{user_id}:{path}"
    return urn

json_call["input_data"] = {
    "image": upload_file('ceph.jpg')
}

# 提交请求

使用 `/run` POST方法提交请求

In [ ]:
headers = {
  "Content-Type": "application/json",
  "X-ZH-TOKEN": zh_token
}

url = base_url + '/run'

response = requests.request("POST", url, headers=headers, data=json.dumps(json_call))
response.raise_for_status()
create_result = response.json()
run_id = create_result['run_id']

# 打印任务号
print(run_id)

# 等待请求完成

使用轮询API等待请求完成，API地址为 `/run/{run_id}`

**强烈推荐使用回调方法**, 参考[定义回调地址](#定义回调地址)，回调方法无论工作流成功与否，在结束时均会向给定地址发送回调信息，您无需轮询结果

In [ ]:
# 第二步： 轮询状态
url = base_url + f"/run/{run_id}"

start_time = time.time()
while time.time()-start_time < 180: # 最多等3分钟
    time.sleep(0.3) # 轮询间隔
    response = requests.request("GET", url, headers=headers)
    result = response.json()
    if result['completed'] or result['failed']:
        break
    
if not result['completed']:
    if result['failed']:
        raise ValueError("API运行错误，原因： " + str(result['reason_public']))
    raise TimeoutError("API超时")

print("API运行时间： {}s".format(time.time()-start_time))

# 获取请求结果

使用 `/data/{run_id}` 获得该工作流所有输出数据，使用 `/data/{run_id}/{key}` 获取某一项数据

In [ ]:
url = base_url + f"/data/{run_id}"
response = requests.request("GET", url, headers=headers)
result = response.json()

# 样例API输出结果展示

以下为该侧位片分析API的输出结果展示。该部分之上代码对所有以二维图片作为输入的API通用，以下部分仅针对侧位片分析输出处理及展示。

侧位片接口具体输出规则及字段见: https://www.chohotech.com/docs/cloud-zh/#/module/ceph-1

In [ ]:
def vis_teeth(img, pred):
    kps = pred['kps']
    all_pts = []
    all_names = []
    vis = img.copy()
    for name, kp in kps.items():
        x, y = int(kp[0]), int(kp[1])
        cv2.circle(vis, (x, y), radius=3, color=(255, 255, 255), thickness=-1)
        all_pts.append(kp)
        all_names.append(name)
    for kp in json.loads(pred['meta'])['lower_jaw_contour']:
        x, y = int(kp[0]), int(kp[1])
        cv2.circle(vis, (x, y), radius=3, color=(255, 0, 0), thickness=-1)
    
    target_names = [
        ['100800', '100040', '100802', '100803', '100041', '100805', '100300', '100807', '100808', '100054', '100810', '100053', '100812', '100042', '100814', '100055', '100043', '100045', '100818', '100050'],
        ['100049', '100821', '100046', '100044', '100824', '100051', '100826', '100827', '100047', '100829', '100052', '100831', '100048', '100833', '100834', '100056', '100836'],
        ['100019', '100018', '100839', '100840', '100020', '100022', '100021', '100844', '100845', '100303', '100027', '100017', '100026', '100850', '100851', '100016', '100025', '100015', '100855', '100856', '100857', '100858', '100859', '100860', '100861', '100862'],
        ['100302', '100864', '100865', '100866', '100867', '100021'],
        ['100008', '100870', '100013', '100872', '100873', '100874', '100008'],
        ['100301', '100876', '100877', '100878', '100879', '100880', '100010', '100882', '100883', '100884', '100885', '100886', '100887', '100009', '100889', '100011', '100891', '100012'],
        ['100893', '100894', '100003', '100896', '100897', '100898', '100899', '100003'],
        ['100901', '100902', '100903', '100904', '100905', '100906', '100907', '100908', '100909', '100910', '100911', '100912', '100913', '100914', '100005', '100016', '100917', '100918', '100919', '100920', '100902', '100922'],
        ['100004', '100924', '100925', '100926', '100004'],
        ['100904', '100927', '100920', '100929'],
        ['100930', '100007', '100932', '100933', '100934'],
        ['100103', '100936', '100104', '100938', '100939', '100940', '100941', '100942', '100103'],
        ['100220', '100221', '100222', '100223', '100224', '100225', '100226', '100227', '100228', '100229', '100230', '100231', '100232', '100233', '100234', '100235', '100220']
    ]
    
    for target_list in target_names:
        for name in target_list[:-1]:
            next_name = target_list[target_list.index(name) + 1]
            if name in all_names and next_name in all_names:
                idx1 = all_names.index(name)
                idx2 = all_names.index(next_name)
                x1, y1 = int(all_pts[idx1][0]), int(all_pts[idx1][1])
                x2, y2 = int(all_pts[idx2][0]), int(all_pts[idx2][1])
                cv2.line(vis, (x1, y1), (x2, y2), color=(0,255,0), thickness=1)
    return vis

In [ ]:
img = cv2.imread('../../data/ceph.jpg')

In [ ]:
vis = vis_teeth(img, result['result'])

In [ ]:
plt.figure(figsize=(20, 14), dpi=80)
plt.imshow(vis[:,:,::-1])
plt.show()